## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
 
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.

one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = one_hot.transform(y_test.reshape(-1,1)).toarray()

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
from keras.applications.vgg19 import VGG19
input_tensor = Input(shape=(32, 32, 3))

model_vgg = VGG19(include_top=False, weights='imagenet',
                  input_tensor=input_tensor,classes=10)
train_layer = len(model_vgg.layers)
print(train_layer)
x = model_vgg.output

x = Flatten()(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.1)(x)

x = Dense(100, activation='relu')(x)
x = Dropout(0.1)(x)
out = Dense(10, activation='softmax')(x)

model_vgg = Model(inputs=model_vgg.input, outputs=out)

for layer in model_vgg.layers[:train_layer]:
    layer.trainable = False
for layer in model_vgg.layers[train_layer:]:
    layer.trainable = True
model_vgg.summary()

22
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
_______

In [4]:
model_vgg.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_vgg.fit(x_train, y_train, batch_size=32, epochs=20,verbose=1,shuffle=True)

Epoch 1/20
50000/50000 [==============================] - 45s 905us/step - loss: 1.4369 - acc: 0.4915
Epoch 2/20
50000/50000 [==============================] - 44s 872us/step - loss: 1.2589 - acc: 0.5578
Epoch 3/20
50000/50000 [==============================] - 44s 873us/step - loss: 1.1845 - acc: 0.5831
Epoch 4/20
50000/50000 [==============================] - 44s 870us/step - loss: 1.1355 - acc: 0.5978
Epoch 5/20
50000/50000 [==============================] - 44s 877us/step - loss: 1.0922 - acc: 0.6166
Epoch 6/20
50000/50000 [==============================] - 44s 875us/step - loss: 1.0547 - acc: 0.6285
Epoch 7/20
50000/50000 [==============================] - 44s 876us/step - loss: 1.0178 - acc: 0.6407
Epoch 8/20
50000/50000 [==============================] - 44s 878us/step - loss: 0.9840 - acc: 0.6513
Epoch 9/20
50000/50000 [==============================] - 44s 878us/step - loss: 0.9498 - acc: 0.6641
Epoch 10/20
50000/50000 [==============================] - 44s 875us/step - loss: 

In [5]:
def predict_label(predict_val):
    max_label = []
    for i in range(predict_val.shape[0]):
        for j in range(predict_val.shape[1]):
            if predict_val[i,j] == predict_val[i,:].max():
                max_label.append(j)
    return np.array(max_label)

In [7]:
from sklearn.metrics import accuracy_score
predict = model_vgg.predict(x_test)
predict = predict_label(predict)
y_test = one_hot.inverse_transform(y_test)
accuracy_score(y_test, predict)

0.61